## Environment Setup

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/Cel/Documents/Data Analytics/6. Maine Fisheries Landings'

In [28]:
modern_df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'Maine_Modern_Landings_2008-2022_2023-10-13.csv'))

## Cleaning 'modern_df' Data Frame

#### Cleaning up columns

In [29]:
# check shape
modern_df.shape

(4891, 10)

In [30]:
# check columns
modern_df.columns

Index(['year', 'species', 'port', 'county', 'lob_zone', 'weight_type',
       'weight', 'value', 'trip_n', 'harv_n'],
      dtype='object')

In [31]:
# delete 'lob_zone' and 'trip_n' columns
modern_df.drop(columns = ['lob_zone','trip_n'], inplace = True)

In [32]:
modern_df.head()

,year,species,port,county,weight_type,weight,value,harv_n
0,2008,Bloodworms,Addison,Washington,Live Pounds,18934.19,208982.04,145
1,2008,Bloodworms,Bar Harbor,Hancock,Live Pounds,1396.62,15276.99,31
2,2008,Bloodworms,Bass Harbor,Hancock,Live Pounds,42.09,448.58,3
3,2008,Bloodworms,Bath,Sagadahoc,Live Pounds,9408.14,101804.90,60
4,2008,Bloodworms,Beals,Washington,Live Pounds,1760.18,18989.88,75


In [33]:
# rename 'harv_n'
modern_df.rename(columns = {'harv_n' : 'number_harvesters'}, inplace = True)

In [34]:
# check columns
modern_df.head()

,year,species,port,county,weight_type,weight,value,number_harvesters
0,2008,Bloodworms,Addison,Washington,Live Pounds,18934.19,208982.04,145
1,2008,Bloodworms,Bar Harbor,Hancock,Live Pounds,1396.62,15276.99,31
2,2008,Bloodworms,Bass Harbor,Hancock,Live Pounds,42.09,448.58,3
3,2008,Bloodworms,Bath,Sagadahoc,Live Pounds,9408.14,101804.90,60
4,2008,Bloodworms,Beals,Washington,Live Pounds,1760.18,18989.88,75


#### General data consistency checks

In [36]:
modern_df.describe()

,year,weight,value,number_harvesters
count,4891.000000,4.891000e+03,4.891000e+03,4891.000000
mean,2015.076467,7.771021e+05,1.711111e+06,48.544674
std,4.269773,3.081906e+06,5.919887e+06,67.416732
min,2008.000000,0.000000e+00,0.000000e+00,1.000000
25%,2011.000000,5.268425e+03,3.571507e+04,10.000000
50%,2015.000000,5.883733e+04,1.824522e+05,25.000000
75%,2019.000000,3.840468e+05,8.240626e+05,58.000000
max,2022.000000,5.178894e+07,1.001032e+08,791.000000


In [37]:
# create subset to inspect min weight = 0
min_weight = modern_df[modern_df['weight'] == 0]

In [38]:
min_weight

,year,species,port,county,weight_type,weight,value,number_harvesters
4816,2022,Menhaden Unc,Other Maine,Not-Specified,Live Pounds,0.0,0.0,22


In [39]:
# create subset to inspect min value = 0
min_value = modern_df[modern_df['value'] == 0]

In [40]:
min_value

,year,species,port,county,weight_type,weight,value,number_harvesters
4816,2022,Menhaden Unc,Other Maine,Not-Specified,Live Pounds,0.0,0.0,22


In [43]:
# delete index 4816
modern_df = modern_df.drop(4816)

In [44]:
modern_df.shape

(4890, 8)

#### Find and address mixed data types

In [49]:
modern_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4890 entries, 0 to 4890
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               4890 non-null   int64  
 1   species            4889 non-null   object 
 2   port               4890 non-null   object 
 3   county             4890 non-null   object 
 4   weight_type        4890 non-null   object 
 5   weight             4890 non-null   float64
 6   value              4890 non-null   float64
 7   number_harvesters  4890 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 343.8+ KB


In [47]:
# use for-loop to check for columns with mixed data types

for col in modern_df.columns.tolist():
  weird = (modern_df[[col]].applymap(type) != modern_df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (modern_df[weird]) > 0:
    print (col)

species


In [53]:
# convert all cells in 'species' to object data type
modern_df['species'] = modern_df['species'].astype('str')

In [54]:
# double check
modern_df['species'].dtype

dtype('O')

#### Find and address missing values

In [55]:
# find missing values
modern_df.isnull().sum()

year                 0
species              0
port                 0
county               0
weight_type          0
weight               0
value                0
number_harvesters    0
dtype: int64

In [59]:
# check for 'unknown' category names in 'year'
modern_df['year'].value_counts(dropna = False)

year
2015    352
2018    346
2014    342
2016    341
2021    334
2013    331
2012    330
2017    328
2022    328
2020    322
2010    317
2011    311
2008    307
2009    301
2019    300
Name: count, dtype: int64

In [60]:
# check for 'unknown' category names in 'species'
modern_df['species'].value_counts(dropna = False)

species
Lobster American                       1114
Clam Soft                               922
Elver                                   525
Bloodworms                              345
Scallop Sea                             289
Menhaden Atlantic                       225
Oyster Eastern / American               133
Crab Atlantic Rock                      128
Sandworms                               112
Crab Jonah                              108
Sea Urchin                              105
Clam Northern Quahog / Hard             102
Halibut Atlantic                         99
Tuna Atlantic Bluefin                    99
Herring Atlantic                         76
Shrimp Northern                          54
Periwinkle Common                        43
Mussel Blue Sea                          35
Clam Razor Atlantic                      28
Oyster European Flat                     26
Haddock                                  24
Cod Atlantic                             24
Pollock                 

In [61]:
# check for 'unknown' category names in 'county'
modern_df['county'].value_counts(dropna = False)

county
Hancock          1251
Washington        896
Cumberland        626
Knox              588
Lincoln           509
Not-Specified     453
York              205
Sagadahoc         198
Waldo             107
Penobscot          32
UK                 21
Kennebec            4
Name: count, dtype: int64

In [62]:
# check for 'unknown' category names in 'weight_type'
modern_df['weight_type'].value_counts(dropna = False)

weight_type
Live Pounds    4601
Meat Pounds     289
Name: count, dtype: int64

A few flag categories exist in the data: species = 'unidentified catch', and county = 'UK' (unknown) and 'Not-specified'

#### Find and address duplicate records

In [65]:
# create subset for full duplicates
df_dups = modern_df[modern_df.duplicated()]

In [66]:
df_dups

,year,species,port,county,weight_type,weight,value,number_harvesters


No full duplicates!

## Summary statistics after data cleaning

In [99]:
modern_df.describe()

,year,weight,value,number_harvesters
count,4890.000000,4.890000e+03,4.890000e+03,4890.000000
mean,2015.075051,7.772610e+05,1.711461e+06,48.550102
std,4.269061,3.082201e+06,5.920442e+06,67.422557
min,2008.000000,2.000000e-01,2.150000e+00,1.000000
25%,2011.000000,5.274995e+03,3.573835e+04,10.000000
50%,2015.000000,5.888356e+04,1.825623e+05,25.000000
75%,2019.000000,3.844331e+05,8.242147e+05,58.000000
max,2022.000000,5.178894e+07,1.001032e+08,791.000000


In [111]:
modern_df.head()

,year,species,port,county,weight_type,weight,value,number_harvesters
0,2008,Bloodworms,Addison,Washington,Live Pounds,18934.19,208982.04,145
1,2008,Bloodworms,Bar Harbor,Hancock,Live Pounds,1396.62,15276.99,31
2,2008,Bloodworms,Bass Harbor,Hancock,Live Pounds,42.09,448.58,3
3,2008,Bloodworms,Bath,Sagadahoc,Live Pounds,9408.14,101804.90,60
4,2008,Bloodworms,Beals,Washington,Live Pounds,1760.18,18989.88,75


## Export as pkl

In [113]:
# exporting clean data frame
modern_df.to_csv(os.path.join(path, '02 Data','Prepared Data', 'landings_clean.csv'))